In [1]:
#importing the required packages
import requests
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import date,timedelta

In [2]:
def retrieving_information_for_the_day(year,month,day): 
    # iterate the dates to get news from these dates
    web_url= "https://www.thehindu.com/archive/web/"+str(year)+"/"+str(month)+"/"+str(day)+"/"
    #getting html source code from the url
    raw_data=requests.get(web_url,'lxml')
    #We create a Beautiful soup object and use it to find the required information from the url
    soup=bs(raw_data.content,'html5lib')
    information=soup.findAll('ul',class_='archive-list')
    return information

In [3]:
#the headlines from the day
def news(information):
    news=[]
    for i in range(len(information)):
        for j in range(len(information[i].findAll('a'))):
              news.append(information[i].findAll('a')[j])
    return news

In [4]:
#this is a function to scrap the title of the news and the link to the main content from the information
def news_links(headlines):
    links=[]
    for i in range(len(headlines)):
        links.append(headlines[i]['href'])
    return links

In [5]:
#this function scraps the authors of the articles and  identifies the articles with no authors.
#These articles are then removed.
def news_authors(links):
    authors=[]
    noauthors=[]
    c=0
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        author=soup1.find('a',{"class":["auth-nm lnk","auth-nm no-lnk"]})
        if author!=None:
            authors.append(author.get_text().strip('\n'))
        if author==None:
            noauthors.append(i)
            authors.append(0)
    return authors,noauthors

In [6]:
def processing(authors,noauthors,links,headlines):
    c=0
    for i in noauthors:
        authors.remove(authors[i-c])
        links.remove(links[i-c])
        headlines.remove(headlines[i-c])
        c=c+1
    return authors,links,headlines

In [7]:
#this is a function to scrap the title of the news and the link to the main content from the information
def newstitles(headlines):
    news_titles=[]
    for i in range(len(headlines)):
        news_titles.append(headlines[i].get_text())
    return news_titles

In [8]:
#This function finds the date and time of publishing of the article.
def date_and_time(links):
    date=[]
    time=[]
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        when=soup1.find('meta',{"name":"publish-date"})
        date_and_time=when['content'].split('T')
        date.append(date_and_time[0])
        time.append(date_and_time[1].replace("+05:30"," IST"))
    return date,time

In [9]:
#this function goes to the links of the headines and gets the full content
def retrieving_news_content(links):
    news_contents=[]
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        raw_content=soup1.findAll('p',class_="")
        paragraph=''
        for j in range(len(raw_content)):
            if len(raw_content[j].get_text())>10:
                paragraph+=raw_content[j].get_text()
        news_contents.append(paragraph.strip('\n').split("Start your 14 days trial now.")[0])
    return news_contents

Change the dates to increase the number of articles scrapped.

In [ ]:
all_links=[]
all_news_titles=[]
all_authors=[]
all_noauthors=[]
all_news_contents=[]
all_dates=[]
all_times=[]

In [23]:
#Enter the start and end date for news articles in yyyy-mm-dd format.
start_date=date(2010,5,7)
end_date=date(2010,5,8)
time_difference=(end_date-start_date).days

In [24]:
#This is the function to retrieve the articles and make a corpus.
def news_articles(time_difference):
    for i in range(0,time_difference):
        year=(start_date+timedelta(i)).year
        month=(start_date+timedelta(i)).month
        day=(start_date+timedelta(i)).day
        information=retrieving_information_for_the_day(year,month,day)
        headlines=news(information)
        links=news_links(headlines)
        authors,noauthors=news_authors(links)
        authors,links,headlines=processing(authors,noauthors,links,headlines)
        all_authors.extend(authors)
        all_links.extend(links)
        news_titles=newstitles(headlines)
        all_news_titles.extend(news_titles)
        date,time=date_and_time(links)
        all_dates.extend(date)
        all_times.extend(time)
        news_contents=retrieving_news_content(links)
        all_news_contents.extend(news_contents)
    return all_links,all_news_titles,all_authors,all_news_contents,all_dates,all_times

In [25]:
# We could take a long enough time span for making the corpus
#or we could take short steps of 5 days each and make a large corpus
all_links,all_news_titles,all_authors,all_news_contents,all_dates,all_times=news_articles(time_difference)

In [26]:
#In case you are making the corpus step by step
all_links.extend(all_links)
all_news_titles.extend(all_news_titles)
all_authors.extend(all_authors)
all_news_contents.extend(all_news_contents)
all_dates.extend(all_dates)
all_times.extend(all_times)

In [27]:
data={'news titles':all_news_titles,'author':all_authors,'date':all_dates,'time':all_times,'content':all_news_contents,'links':all_links}

In [28]:
corpus=pd.DataFrame(data)
corpus

,news titles,author,date,time,content,links
0,Second Thoughts: Come with old Khayyam,NAVTEJ SARNA,2010-05-06,18:56:53 IST,"The other day, halfway through a virtual conve...",https://www.thehindu.com/opinion/columns/navte...
1,Mystery Knight bags feature,Racing Correspondent,2010-05-06,16:52:09 IST,Mr.M.A.M.R.Muthiah’s Mystery Knight (Manikanda...,https://www.thehindu.com/sport/races/Mystery-K...
2,Sky Drive shows out,Racing Correspondent,2010-05-06,14:45:21 IST,"Sky Drive, Cumulus Nimbus, Stormyred and Athab...",https://www.thehindu.com/sport/races/Sky-Drive...
3,Staging a change,SANGEETA BAROOAH PISHAROTY,2010-05-06,17:10:30 IST,Sitanath Lahkar has made his passion blossom. ...,https://www.thehindu.com/features/friday-revie...
4,In Ghalib's skin,DIWAN SINGH BAJELI,2010-05-06,16:49:22 IST,"“In recent years, we have seen several plays a...",https://www.thehindu.com/features/friday-revie...
...,...,...,...,...,...,...
1395,Carbon black plant opposed,Special Correspondent,2010-05-07,00:19:55 IST,Almost all those who attended the environmenta...,https://www.thehindu.com/news/cities/Visakhapa...
1396,Kuchipudi dance team floored by response in US,Staff Reporter,2010-05-07,00:17:55 IST,Presdent of Rotary Club Elite Ali speaking at ...,https://www.thehindu.com/news/cities/Visakhapa...
1397,Chill with apricots,Chef NARAYAN RAO,2010-05-07,20:19:39 IST,"Khubani Ka Meetha at AMAN Hotel, New Delhi. Ph...",https://www.thehindu.com/life-and-style/Chill-...
1398,Vettel sets pace in Spain,DPA,2010-05-07,20:24:36 IST,Red Bull F1 driver Sebastian Vettel\n The Red...,https://www.thehindu.com/sport/motorsport/Vett...


In [29]:
corpus.to_csv('corpus.csv')